# Image AR Project for MyData

- Start date : 2022.01.18
- Author : Hyoungjoon Lim
- Image crawling for scene text classification (200 images per class)

## 1. Import packages

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import urllib.request
import os
import xml.etree.ElementTree as ET
import splitfolders
import glob
import pandas as pd

## 2. Input keywords (list)

In [7]:
keywords = ['우리은행 뉴스']

## 3. Crawler

In [3]:
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

def crawling_img(name):
    driver = webdriver.Chrome()
    driver.get("https://www.google.co.kr/imghp?hl=ko&tab=wi&authuser=0&ogbl")
    elem = driver.find_element_by_name("q")
    elem.send_keys(name)
    elem.send_keys(Keys.RETURN)

    #
    SCROLL_PAUSE_TIME = 1
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")  # 브라우저의 높이를 자바스크립트로 찾음
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # 브라우저 끝까지 스크롤을 내림
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            try:
                driver.find_element_by_css_selector(".mye4qd").click()
            except:
                break
        last_height = new_height

    imgs = driver.find_elements_by_css_selector(".rg_i.Q4LuWd")
    dir = ".\crawled_image" + "\\" + name

    createDirectory(dir) #폴더 생성
    count = 1
    for img in imgs:
        try:
            img.click()
            time.sleep(2)
            imgUrl = driver.find_element_by_xpath(
                '//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div[1]/a/img').get_attribute(
                "src")
            path = "C:\\Users\\Hyoungjoon LIM\\Desktop\\scene_text_classification\\crawled_image\\" + name + "\\"
            urllib.request.urlretrieve(imgUrl, path + name + str(count) + ".jpg")
            count = count + 1
            if count > 200:
                break
        except:
            pass
    driver.close()


In [8]:
for keyword in keywords:
    crawling_img(keyword)

## 4. Rename .jpg files

In [2]:
path = 'C:\\ProgramData\\Anaconda3\\envs\\tf1\\Lib\\site-packages\\tensorflow\\models\\image\\'
counter = 1
for f in os.listdir(path):
    print(f)
    suffix = f.split('.')[-1]
    if suffix == 'jpg' or suffix == 'png':
        new = '{}.{}'.format(str(counter), suffix)
        os.rename(path + f, path + new)
        counter = int(counter) + 1

#### 4.1. If you wanna add sth...

In [2]:
path = r'C:\Users\Hyoungjoon LIM\Desktop\scene_text_classification\직원추가\\'
counter = 4142 # start_number
for f in os.listdir(path):
    print(f)
    suffix = f.split('.')[-1]
    if suffix == 'jpg' or suffix == 'png' or suffix == 'jpeg':
        new = '{}.{}'.format(str(counter), suffix)
        os.rename(path + f, path + new)
        counter = int(counter) + 1

KakaoTalk_20220214_145842518.jpg
KakaoTalk_20220214_145842518_01.jpg
KakaoTalk_20220214_145842518_02.jpg
KakaoTalk_20220214_145842518_03.jpg
KakaoTalk_20220214_145842518_04.jpg
KakaoTalk_20220214_145842518_05.jpg
KakaoTalk_20220214_145842518_06.jpg
KakaoTalk_20220214_145842518_07.jpg
KakaoTalk_20220214_145842518_08.jpg
KakaoTalk_20220214_145842518_09.jpg
KakaoTalk_20220214_145842518_10.jpg
KakaoTalk_20220214_145842518_11.jpg
KakaoTalk_20220214_145842518_12.jpg
KakaoTalk_20220214_145842518_13.jpg
KakaoTalk_20220214_145842518_14.jpg
KakaoTalk_20220214_145842518_15.jpg
KakaoTalk_20220214_145842518_16.jpg
KakaoTalk_20220214_145842518_17.jpg
KakaoTalk_20220214_145842518_18.jpg
KakaoTalk_20220214_145842518_19.jpg
KakaoTalk_20220214_145842518_20.jpg
KakaoTalk_20220214_145842518_21.jpg
KakaoTalk_20220214_145842518_22.jpg
KakaoTalk_20220214_145842518_23.jpg
KakaoTalk_20220214_145842518_24.jpg
KakaoTalk_20220214_145842518_25.jpg
KakaoTalk_20220214_145842518_26.jpg
KakaoTalk_20220214_145842518_27

## 5. Data preparation

### 5.1 Image split (70%, 30%)

In [14]:
splitfolders.ratio(r"C:\Users\Hyoungjoon LIM\Desktop\scene_text_classification\yolo_label\all_data", 
                    output=r"C:\Users\Hyoungjoon LIM\Desktop\scene_text_classification\yolo_label\all_data",
                    seed=19, ratio=(.7,.3), group_prefix=2)

Copying files: 1625 files [01:02, 25.83 files/s]


In [15]:
traindir = r"C:\Users\Hyoungjoon LIM\Desktop\scene_text_classification\yolo_label\all_data\train\class"
testdir = r"C:\Users\Hyoungjoon LIM\Desktop\scene_text_classification\yolo_label\all_data\val\class"

train_list,test_list = os.listdir(traindir), os.listdir(testdir)

In [18]:
print(int(len(train_list)/2), int(len(test_list)/2))

1137 488


### 5.2 Modifying xml directory

In [2]:
targetdir1 = r"C:\Users\Hyoungjoon LIM\Desktop\scene_text_classification\xml" # XML dir

file_list = os.listdir(targetdir1)
xml_list = []
for file in file_list:
    if '.xml' in file:
        xml_list.append(file)

#### 5.2.1. xml dir initialization

In [3]:
for xml_file in xml_list:
    target_path = targetdir1 + "\\" + xml_file
    targetXML = open(target_path, 'rt', encoding='UTF8')

    tree = ET.parse(targetXML)

    root = tree.getroot()

    ##수정할 부분

    folder_tag = root.find("folder")
    path_tag = root.find("path")
    name_tag = root.find("filename")
    name = name_tag.text
    
    path_tag.text = r"C:\ProgramData\Anaconda3\envs\tf1\Lib\site-packages\tensorflow\models\image\class"+"\\"+name  #수정
      
    tree.write(target_path)

#### 5.2.2. modifying train/test directory

In [8]:
train_xml, test_xml = 0, 0

for xml_file in xml_list:
    target_path = targetdir1 + "\\" + xml_file
    targetXML = open(target_path, 'rt', encoding='UTF8')

    tree = ET.parse(targetXML)

    root = tree.getroot()

    ##수정할 부분
    name_tag = root.find("filename")
    name = name_tag.text
    folder_tag = root.find("folder")
    path_tag = root.find("path")
    
    if name in train_list:
        original = path_tag.text     #원본 String
        modified = original.replace(r"C:\ProgramData\Anaconda3\envs\tf1\Lib\site-packages\tensorflow\models\image\class",
                                    "/content/drive/MyDrive/ARmodel/models/image/train/class")
        modified = modified.replace("\\", "/")

        path_tag.text = modified  #수정

        train_xml += 1
    elif name in test_list:
        original = path_tag.text     #원본 String
        modified = original.replace(r"C:\ProgramData\Anaconda3\envs\tf1\Lib\site-packages\tensorflow\models\image\class",
                                    "/content/drive/MyDrive/ARmodel/models/image/val/class")
        modified = modified.replace("\\", "/")

        path_tag.text = modified  #수정

        test_xml += 1
    else: continue
      
    tree.write(target_path)
    print(train_xml, test_xml)

1 0
2 0
3 0
4 0
5 0
6 0
7 0
7 1
8 1
9 1
9 2
10 2
11 2
12 2
13 2
14 2
15 2
15 3
15 4
16 4
17 4
18 4
18 5
18 6
19 6
19 7
19 8
20 8
21 8
22 8
23 8
24 8
24 9
25 9
26 9
27 9
27 10
28 10
29 10
30 10
30 11
31 11
32 11
32 12
33 12
34 12
35 12
36 12
37 12
38 12
39 12
40 12
41 12
42 12
43 12
44 12
45 12
45 13
46 13
47 13
48 13
49 13
50 13
51 13
51 14
52 14
53 14
54 14
54 15
55 15
55 16
56 16
56 17
57 17
58 17
59 17
59 18
60 18
61 18
61 19
62 19
63 19
64 19
65 19
66 19
67 19
68 19
69 19
69 20
70 20
71 20
71 21
71 22
72 22
72 23
72 24
72 25
73 25
74 25
75 25
76 25
77 25
77 26
78 26
79 26
80 26
81 26
82 26
83 26
83 27
84 27
85 27
86 27
86 28
87 28
88 28
88 29
88 30
89 30
90 30
90 31
91 31
91 32
91 33
92 33
92 34
93 34
93 35
93 36
94 36
95 36
96 36
97 36
98 36
99 36
100 36
101 36
102 36
103 36
104 36
105 36
106 36
107 36
108 36
109 36
110 36
111 36
111 37
112 37
113 37
114 37
115 37
115 38
116 38
116 39
117 39
118 39
119 39
120 39
121 39
121 40
121 41
122 41
123 41
124 41
124 42
125 42
125 43
125 44

### 5.3. Delete non-RGB images

In [9]:
from PIL import Image     

train_del, test_del = [], []

for file in train_list:      
    extension = file.split('.')[-1]
    fileLoc = traindir+"\\"+file
    with Image.open(fileLoc) as img:
        mode = img.mode
    if mode != 'RGB':
        train_del.append(file)
        os.remove(fileLoc)
        
for file in test_list:      
    extension = file.split('.')[-1]
    fileLoc = testdir+"\\"+file
    with Image.open(fileLoc) as img:
        mode = img.mode
    if mode != 'RGB':
        test_del.append(file)
        os.remove(fileLoc)

In [10]:
print(len(train_del), len(test_del))

224 117


In [11]:
xml_del = train_del + test_del
xml_del_m = []

for i in xml_del:
    temp = i.replace('.jpg','.xml')
    temp = temp.replace('.png','.xml')
    temp = temp.replace('.jpeg','.xml')
    xml_del_m.append(temp)
    
for file in os.listdir(targetdir1):
    fileLoc = targetdir1+"\\"+file
    if file in xml_del_m:
        os.remove(fileLoc)

In [12]:
traindir = r"C:\ProgramData\Anaconda3\envs\tf1\Lib\site-packages\tensorflow\models\image\train\class"
testdir = r"C:\ProgramData\Anaconda3\envs\tf1\Lib\site-packages\tensorflow\models\image\val\class"

all_list = os.listdir(traindir) + os.listdir(testdir)

targetdir1 = r"C:\ProgramData\Anaconda3\envs\tf1\Lib\site-packages\tensorflow\models\annotations\xmls" # XML dir

file_list = os.listdir(targetdir1)
xml_list = []
for file in file_list:
    if '.xml' in file:
        xml_list.append(file)

for xml_file in xml_list:
    target_path = targetdir1 + "\\" + xml_file
    with open(target_path, 'rt', encoding='UTF8') as targetXML:
        tree = ET.parse(targetXML)
    root = tree.getroot()
    name_tag = root.find("filename")
    name = name_tag.text
    if name not in all_list:
        os.remove(target_path)

### 5.4. xml to csv

In [13]:
def xml_to_csv(path):
    xml_train_list = []
    xml_test_list = []

    for xml_file in glob.glob(path + '\\*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        path_tag = root.find("path")
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            if 'train' in path_tag.text:
                xml_train_list.append(value)
            else:
                xml_test_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_train_df, xml_test_df = pd.DataFrame(xml_train_list, columns=column_name), pd.DataFrame(xml_test_list, columns=column_name)
    return xml_train_df, xml_test_df

xml_train_df, xml_test_df = xml_to_csv(targetdir1)
xml_train_df.to_csv(r'C:\ProgramData\Anaconda3\envs\tf1\Lib\site-packages\tensorflow\models\data\train_labels.csv', index=None)
xml_test_df.to_csv(r'C:\ProgramData\Anaconda3\envs\tf1\Lib\site-packages\tensorflow\models\data\test_labels.csv', index=None)

print(xml_train_df.shape[0], xml_test_df.shape[0])

571 224


### 5.5. Create tfrecord (@conda cmd)

(tf1) C:\ProgramData\Anaconda3\envs\tf1\Lib\site-packages\tensorflow\models>

```
python research\object_detection\dataset_tools\generate_tfrecord.py --csv_input=data\train_labels.csv --output_path=train.record --image_dir=image\train\class

python research\object_detection\dataset_tools\generate_tfrecord.py --csv_input=data\test_labels.csv --output_path=test.record --image_dir=image\val\class
```

## 6. Train (@conda cmd)

```
activate tf1

cd C:\ProgramData\Anaconda3\envs\tf1\Lib\site-packages\tensorflow\models

python research\object_detection\model_main.py --pipeline_config_path=ssd_mobilenet_v2_coco.config --model_dir=train --num_train_steps=20000 --sample_1_of_n_eval_examples=1 --alsologtostderr
```

In [12]:
image_dir = r"C:\Users\Hyoungjoon LIM\Desktop\scene_text_classification\paired_image"

image_list = os.listdir(image_dir)

xml_dir = r"C:\Users\Hyoungjoon LIM\Desktop\scene_text_classification\all_xml" # XML dir

file_list = os.listdir(xml_dir)
xml_list = []

for file in file_list:
    xml_path = xml_dir + "\\" + file
    with open(xml_path, 'rt', encoding='UTF8') as targetXML:
        tree = ET.parse(targetXML)
    root = tree.getroot()
    name_tag = root.find("filename")
    name = name_tag.text
    xml_list.append(name)

In [13]:
for image in image_list:
    target_path = image_dir + "\\" + image
    if image not in xml_list:
        os.remove(target_path)